In [ ]:
import pandas as pd

# 1. قراءة الملف
df = pd.read_csv('../data/raw/sleep_data_raw.csv', encoding='cp1256')

# 2. تنظيف أسماء الأعمدة
# تنظيف أسماء الأعمدة من الفراغات والرموز الخفية مثل \xa0
df.columns = df.columns.str.strip().str.replace('\xa0', '', regex=False)

# إعادة التسمية لتسهيل الاستخدام
df.rename(columns={
    'كم ساعة نمت الليلة الماضية ؟': 'ساعات_النوم',
    'هل شعرت برغبة في تأجيل مهامك أو التسويف؟': 'تسويف',
    'قدرتك على التركيز الذهني اليوم؟': 'تركيز',
    'كيف تقيم مستوى طاقتك الآن؟': 'طاقة',
    'كيف تقيم مزاجك اليوم؟': 'مزاج',
    'هل سبق ولاحظت أن النوم الجيد يحسن حالتك النفسية؟': 'تأثير_النوم_على_النفسية',
    'هل واجهت صعوبة في التعامل مع الآخرين اليوم؟': 'صعوبة_اجتماعية'
}, inplace=True)


# 4. تحويل "ساعات_النوم" إلى أرقام
mapping_sleep = {
    'اقل من 4': 3,
    '4-6 ساعات': 5,
    '6-8 ساعات': 7,
    'أكثر من 8 ساعات': 9
}
df['ساعات_النوم_رقمي'] = df['ساعات_النوم'].map(mapping_sleep)

# 5. تحويل "تسويف" إلى أرقام
mapping_yes_no = {
    'نعم': 1,
    'لا': 0,
    'الئ حد ما': 0.5,
    'إلى حد ما': 0.5
}
df['تسويف_رقمي'] = df['تسويف'].map(mapping_yes_no)

# 6. تحويل الأعمدة الرقمية (مزاج، طاقة، تركيز) إلى أرقام
for col in ['مزاج', 'طاقة', 'تركيز']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 7. معالجة القيم المفقودة
df['ساعات_النوم_رقمي'].fillna(df['ساعات_النوم_رقمي'].median(), inplace=True)
df['تسويف_رقمي'].fillna(df['تسويف_رقمي'].median(), inplace=True)
df['مزاج'].fillna(df['مزاج'].median(), inplace=True)
df['طاقة'].fillna(df['طاقة'].median(), inplace=True)
df['تركيز'].fillna(df['تركيز'].median(), inplace=True)

# 8. حذف الأعمدة الفارغة تمامًا أو الغير مفيدة (اختياري)
df.drop(columns=['البريد الإلكتروني', 'الاسم'], inplace=True, errors='ignore')

# 9. حفظ الملف النظيف (اختياري)
# حفظ الملف النظيف داخل مجلدك الحالي
df.to_csv(r'C:\Users\gknow\Desktop\sleep_study_project\data\processed\sleep_data_clean.csv', index=False, encoding='utf-8-sig')


# 10. عرض معاينة
df.head()


In [ ]:
df.columns = df.columns.str.strip() 

In [14]:
print(df['هل تستخدم الجوال قبل النوم؟'].unique())


['أقل من 15 دقيقة' ' لا' 'من 15–30 دقيقة' 'أكثر من 30 دقيقة']


In [15]:
# تنظيف الفراغات من القيم
df['هل تستخدم الجوال قبل النوم؟'] = df['هل تستخدم الجوال قبل النوم؟'].str.strip()

# اعتبر "لا" فقط هي عدم استخدام الجوال، والباقي مستخدمين
phone_users_mood = df[df['هل تستخدم الجوال قبل النوم؟'] != 'لا']['المزاج'].dropna()
no_phone_mood = df[df['هل تستخدم الجوال قبل النوم؟'] == 'لا']['المزاج'].dropna()

print("حجم عينات مستخدمي الجوال:", phone_users_mood.count())
print("حجم عينات غير مستخدمي الجوال:", no_phone_mood.count())

# اختبار t إذا الحجم مناسب
if phone_users_mood.count() > 1 and no_phone_mood.count() > 1:
    t_stat, p_value = stats.ttest_ind(phone_users_mood, no_phone_mood)
    diff = no_phone_mood.mean() - phone_users_mood.mean()
    print(f"الفرق في المزاج بين مستخدمي الجوال وغيرهم: {diff:.1f} نقطة")
    print(f"قيمة p لاختبار t: {p_value:.4f}")
else:
    print("عدد البيانات في إحدى المجموعات قليل جداً لإجراء اختبار t.")


حجم عينات مستخدمي الجوال: 74
حجم عينات غير مستخدمي الجوال: 5
الفرق في المزاج بين مستخدمي الجوال وغيرهم: 0.4 نقطة
قيمة p لاختبار t: 0.4567


In [16]:
from statsmodels.stats.power import TTestIndPower

# إعداد الحساب
analysis = TTestIndPower()
effect_size = (phone_users_mood.mean() - no_phone_mood.mean()) / df['المزاج'].std()
power = analysis.solve_power(effect_size=effect_size, nobs1=len(phone_users_mood), ratio=len(no_phone_mood)/len(phone_users_mood), alpha=0.05)

print(f"حجم التأثير (Effect size): {effect_size:.3f}")
print(f"القوة الإحصائية (Power): {power:.3f}")


حجم التأثير (Effect size): -0.347
القوة الإحصائية (Power): 0.115
